# Méthode KNN

La méthode KNN est un algorithme d'apprentissage supervisé. Il vise à …

Dans un 1er temps nous allons importer les données nécessaires.

# Initialisation 
Initialisation des bibliothèques nécessaires

In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

# Récupération des données

In [2]:
data_directory = '../input/gem-predictive-maintenance-2022'
chosen_dataset = 'FD003'

df_train = pd.read_csv(os.path.join(data_directory, chosen_dataset + '_df_train.csv'), index_col=0) 
df_test = pd.read_csv(os.path.join(data_directory, chosen_dataset + '_df_test.csv'), index_col=0)

df_train

,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27
0,1,1,-0.0005,0.0004,100.0,518.67,642.36,1583.23,1396.84,14.62,...,2388,100.0,39.11,23.3537,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,0.0008,-0.0003,100.0,518.67,642.50,1584.69,1396.89,14.62,...,2388,100.0,38.99,23.4491,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,-0.0014,-0.0002,100.0,518.67,642.18,1582.35,1405.61,14.62,...,2388,100.0,38.85,23.3669,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,-0.0020,0.0001,100.0,518.67,642.92,1585.61,1392.27,14.62,...,2388,100.0,38.96,23.2951,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,0.0016,0.0000,100.0,518.67,641.68,1588.63,1397.65,14.62,...,2388,100.0,39.14,23.4583,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24715,100,148,-0.0016,-0.0003,100.0,518.67,643.78,1596.01,1424.11,14.62,...,2388,100.0,38.44,22.9631,NaN,NaN,NaN,NaN,NaN,NaN
24716,100,149,0.0034,-0.0003,100.0,518.67,643.29,1596.38,1429.14,14.62,...,2388,100.0,38.50,22.9746,NaN,NaN,NaN,NaN,NaN,NaN
24717,100,150,-0.0016,0.0004,100.0,518.67,643.84,1604.53,1431.41,14.62,...,2388,100.0,38.39,23.0682,NaN,NaN,NaN,NaN,NaN,NaN
24718,100,151,-0.0023,0.0004,100.0,518.67,643.94,1597.56,1426.57,14.62,...,2388,100.0,38.31,23.0753,NaN,NaN,NaN,NaN,NaN,NaN


# Testing set

In [3]:
df_train.describe()

,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27
count,24720.000000,24720.000000,24720.000000,24720.000000,24720.0,24720.00,24720.000000,24720.000000,24720.000000,2.472000e+04,...,24720.0,24720.0,24720.000000,24720.000000,0.0,0.0,0.0,0.0,0.0,0.0
mean,48.631877,139.077063,-0.000024,0.000005,100.0,518.67,642.457858,1588.079175,1404.471212,1.462000e+01,...,2388.0,100.0,38.988552,23.393024,NaN,NaN,NaN,NaN,NaN,NaN
std,29.348985,98.846675,0.002194,0.000294,0.0,0.00,0.523031,6.810418,9.773178,3.552786e-15,...,0.0,0.0,0.248865,0.149234,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,1.000000,-0.008600,-0.000600,100.0,518.67,640.840000,1564.300000,1377.060000,1.462000e+01,...,2388.0,100.0,38.170000,22.872600,NaN,NaN,NaN,NaN,NaN,NaN
25%,23.000000,62.000000,-0.001500,-0.000200,100.0,518.67,642.080000,1583.280000,1397.187500,1.462000e+01,...,2388.0,100.0,38.830000,23.296200,NaN,NaN,NaN,NaN,NaN,NaN
50%,47.000000,124.000000,0.000000,-0.000000,100.0,518.67,642.400000,1587.520000,1402.910000,1.462000e+01,...,2388.0,100.0,38.990000,23.391600,NaN,NaN,NaN,NaN,NaN,NaN
75%,74.000000,191.000000,0.001500,0.000300,100.0,518.67,642.790000,1592.412500,1410.600000,1.462000e+01,...,2388.0,100.0,39.140000,23.483300,NaN,NaN,NaN,NaN,NaN,NaN
max,100.000000,525.000000,0.008600,0.000700,100.0,518.67,645.110000,1615.390000,1441.160000,1.462000e+01,...,2388.0,100.0,39.850000,23.950500,NaN,NaN,NaN,NaN,NaN,NaN


# Add the rule

In [4]:
def add_rul(g):
    g['RUL'] = max(g['time_in_cycles']) - g['time_in_cycles']
    return g

df_train = df_train.groupby('engine_no').apply(add_rul)
df_test = df_test.groupby('engine_no').apply(add_rul)


print (df_train)

       engine_no  time_in_cycles  op_setting_1  op_setting_2  op_setting_3  \
0              1               1       -0.0005        0.0004         100.0   
1              1               2        0.0008       -0.0003         100.0   
2              1               3       -0.0014       -0.0002         100.0   
3              1               4       -0.0020        0.0001         100.0   
4              1               5        0.0016        0.0000         100.0   
...          ...             ...           ...           ...           ...   
24715        100             148       -0.0016       -0.0003         100.0   
24716        100             149        0.0034       -0.0003         100.0   
24717        100             150       -0.0016        0.0004         100.0   
24718        100             151       -0.0023        0.0004         100.0   
24719        100             152        0.0000        0.0003         100.0   

       sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  ...  s

# Définition des valeurs

In [5]:
#Y :
target_column =['RUL']

#X :
selected_features = [ 'op_setting_1', 'op_setting_2',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_7', 'sensor_8',
       'sensor_9', 'sensor_11', 'sensor_12', 'sensor_13',
       'sensor_14', 'sensor_15', 'sensor_17', 'sensor_20', 'sensor_21']

In [6]:
x_train = df_train[selected_features].values.astype('int')
y_train = df_train[target_column].values.astype('int')

x_test = df_test[selected_features].values.astype('int')
y_test = df_test[target_column].values.astype('int')

print(x_train.shape)
print(y_train)

(24720, 16)
[[258]
 [257]
 [256]
 ...
 [  2]
 [  1]
 [  0]]


In [7]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(x_train, y_train.ravel() )
predict_knn_train = knn.predict(x_train)
predict_knn_test = knn.predict(x_test)
print(y_train,predict_knn_train)

[[258]
 [257]
 [256]
 ...
 [  2]
 [  1]
 [  0]] [ 91 133  58 ...   5  11   0]


In [8]:
df_resultat_train = df_train.assign(RUL_predict = predict_knn_train)
print(df_resultat_train)

df_resultat_test = df_train.assign(RUL_predict = predict_knn_train)
print(df_resultat_test)

       engine_no  time_in_cycles  op_setting_1  op_setting_2  op_setting_3  \
0              1               1       -0.0005        0.0004         100.0   
1              1               2        0.0008       -0.0003         100.0   
2              1               3       -0.0014       -0.0002         100.0   
3              1               4       -0.0020        0.0001         100.0   
4              1               5        0.0016        0.0000         100.0   
...          ...             ...           ...           ...           ...   
24715        100             148       -0.0016       -0.0003         100.0   
24716        100             149        0.0034       -0.0003         100.0   
24717        100             150       -0.0016        0.0004         100.0   
24718        100             151       -0.0023        0.0004         100.0   
24719        100             152        0.0000        0.0003         100.0   

       sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  ...  s

In [9]:
masque = ['engine_no', 'RUL_predict', 'RUL']
df_final_train = df_resultat_train[masque]
print(df_final_train)

       engine_no  RUL_predict  RUL
0              1           91  258
1              1          133  257
2              1           58  256
3              1           61  255
4              1          142  254
...          ...          ...  ...
24715        100            7    4
24716        100            1    3
24717        100            5    2
24718        100           11    1
24719        100            0    0

[24720 rows x 3 columns]


In [10]:
df_final = df_final_train.groupby('engine_no').agg({'RUL_predict': 'first'}).reset_index()
print(df_final)

    engine_no  RUL_predict
0           1           91
1           2           85
2           3          106
3           4           45
4           5           66
..        ...          ...
95         96          142
96         97           77
97         98          100
98         99           62
99        100          179

[100 rows x 2 columns]


In [11]:
df_final.columns = ['engine_no', 'RUL']
df_final[['engine_no', 'RUL']].to_csv('submission.csv', index=False)
df_final

,engine_no,RUL
0,1,91
1,2,85
2,3,106
3,4,45
4,5,66
...,...,...
95,96,142
96,97,77
97,98,100
98,99,62
